In [1]:
from tkinter import *
import math
import datetime
import numpy as np
import time
import pandas as pd
import getpass

In [2]:
def getSEA(latitude = None, longitude = None,  utc_offset = None, date = None, hour = None, minute = None):
    if latitude is None:
        latitude = 55.55
    if longitude is None:
        longitude = 37.37
    if utc_offset is None:
        utc_offset = 3
    date = datetime.datetime.now().timetuple()
    if hour is None:
        hour = date[3]
        #print(hour)
    if minute is None:
        minute = date[4]
        #print(minute)

    hour_minute = (hour + minute / 60) - utc_offset
    day_of_year = date[7]

    g = (360 / 365.25) * (day_of_year + hour_minute / 24)

    g_radians = math.radians(g)

    declination = 0.396372 - 22.91327 * math.cos(g_radians) + 4.02543 * math.sin(g_radians) - 0.387205 * math.cos(
        2 * g_radians) + 0.051967 * math.sin(2 * g_radians) - 0.154527 * math.cos(3 * g_radians) + 0.084798 * math.sin(
        3 * g_radians)

    time_correction = 0.004297 + 0.107029 * math.cos(g_radians) - 1.837877 * math.sin(g_radians) - 0.837378 * math.cos(
        2 * g_radians) - 2.340475 * math.sin(2 * g_radians)

    SHA = (hour_minute - 12) * 15 + longitude + time_correction

    if (SHA > 180):
        SHA_corrected = SHA - 360
    elif (SHA < -180):
        SHA_corrected = SHA + 360
    else:
        SHA_corrected = SHA

    lat_radians = math.radians(latitude)
    d_radians = math.radians(declination)
    SHA_radians = math.radians(SHA)

    SZA_radians = math.acos(
        math.sin(lat_radians) * math.sin(d_radians) + math.cos(lat_radians) * math.cos(d_radians) * math.cos(
            SHA_radians))

    SZA = math.degrees(SZA_radians)

    SEA = 90 - SZA
    return SEA


In [3]:
class Sun:
    def __init__(self, SEA):
        self.r = 1000
        self.x = self.r * math.cos(SEA)
        self.y = self.r * math.sin(SEA)
    def position(self):
        self.x = self.r * math.cos(SEA)
        self.y = self.r * math.sin(SEA)
        position = [self.x,self.y]
        return np.array(position) 
    def is_rise(self):
        if(self.z > 0):
            return True
        else:
            return False
class Flower:
    def __init__(self, x = None, y = None):
        if x is None:
            self.x = 0
        else:
            self.x = x
        if y is None:
            self.y = 0
        else:
            self.y = y
    def position(self):
        return np.array([self.x, self.y])
class Cloude:
    def __init__(self, speed_of_wind = None):
        self.x1 = 500
        self.x2 = 650
        self.y1 = 300
        self.y2 = 300
        if speed_of_wind is None:
            speed_of_wind = (20, 0)
        self.speed_of_wind_x  = speed_of_wind[0]
        self.speed_of_wind_y = speed_of_wind[1]
    def position(self):
        return np.array([self.x1, self.y1, self.x2, self.y2])
    def center_position(self):
        return [(self.x1+self.x2)/2, (self.y1+self.y2)/2]
    def move(self):
        self.y1 = self.y1 + self.speed_of_wind_y
        self.y2 = self.y2 + self.speed_of_wind_y
        self.x1 = self.x1 + self.speed_of_wind_x
        self.x2 = self.x2 + self.speed_of_wind_x
class Lamp:
    def __init__(self):
        self.ligth = False
    def on(self):
        self.ligth = True
        #print("Lamp on")
    def off(self):
        self.ligth = False
        #print("Lamp off")
    def check(self):
        if(self.ligth):
            return "ON"
        else:
            return "OFF"

In [4]:
def update_field(sun, flower, cloude, lamp):
    cloude.move()
    sun.position()
def intersection(sun, cloude, flower): 
        vector_of_sun = sun.position() - flower.position()
        vector_of_cloude = np.array([cloude.position()[2] - cloude.position()[0], cloude.position()[3] - cloude.position()[1]])
        #считаем уравнения прямых проходящих через отрезки
        a1 = -vector_of_sun[1]
        b1 = +vector_of_sun[0]
        d1 = -(a1*sun.x + b1*sun.y)

        a2 = -vector_of_cloude[1]
        b2 = +vector_of_cloude[0]
        d2 = -(a2*cloude.x1 + b2*cloude.y1)

        #подставляем концы отрезков, для выяснения в каких полуплоскотях они
        seg1_line2_start = a2*sun.x + b2*sun.y + d2
        seg1_line2_end = a2*flower.x + b2*flower.y + d2

        seg2_line1_start = a1*cloude.x1 + b1*cloude.y1 + d1
        seg2_line1_end = a1*cloude.x2 + b1*cloude.y2 + d1

        #если концы одного отрезка имеют один знак, значит он в одной полуплоскости и пересечения нет.
        if (seg1_line2_start * seg1_line2_end >= 0 or seg2_line1_start * seg2_line1_end >= 0):
            return False
        return True

In [6]:
try:
    latitude = float(input("Введите широту(Москва - 55.7522): "))
except ValueError:
    latitude = None
try:
    longitude = float(input("Введите долготу(Москва - 37.6156): "))
except ValueError:
    longitude = None
try: 
    utc_offset = int(input("Введите временной сдвиг(Москва - 3 часа): "))
except ValueError:
    utc_offset = None
try:
    hour = int(input("Введите время в часах: "))
except ValueError:
    hour = None
try:
    minute = int(input("Введите время в минутах: "))
except ValueError:
    minute = None

SEA = math.radians(getSEA(latitude, longitude, utc_offset, hour, minute))
sun = Sun(SEA)
cloude = Cloude()
flower = Flower()
lamp = Lamp()
time_on = list()
status = list()
print("Коордитаны солнца: {}".format(sun.position()))
print("Координаты центра облака: {}".format(cloude.center_position()))
for t in range(60):
    time_on.append(t)
    status.append(lamp.check())
    update_field(sun, flower, cloude,lamp)
    if(intersection(sun,cloude,flower)):
        lamp.on()
    else:
        lamp.off()
    #time.sleep(1)
df = pd.DataFrame({'Время с начала запуска': time_on, 'Состояние системы': status})
if(getpass.getuser() == "Snowfall"):
    df.to_excel('Desktop/МАИ/Архитектура информационных систем/ПЗ1/Отчет о работе ламы.xlsx')
else:
    df.to_excel('Desktop/Отчет о работе ламы.xlsx')
df.head(10)

Введите широту(Москва - 55.7522): 
Введите долготу(Москва - 37.6156): 
Введите временной сдвиг(Москва - 3 часа): 
Введите время в часах: 
Введите время в минутах: 
Коордитаны солнца: [ 912.79637388 -408.41496033]
Координаты центра облака: [575.0, 300.0]


,Время с начала запуска,Состояние системы
0,0,OFF
1,1,OFF
2,2,OFF
3,3,OFF
4,4,OFF
5,5,OFF
6,6,OFF
7,7,OFF
8,8,OFF
9,9,OFF
